# Create a preprocessed dataframe containing personas from the persona hub

Data: https://huggingface.co/datasets/proj-persona/PersonaHub

In [1]:
import pandas as pd

In [2]:
import os
from huggingface_hub import login

HF_TOKEN = os.getenv('HF_TOKEN')
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /pfs/work7/workspace/scratch/ma_janjung-master-thesis\token
Login successful


## Load data

In [3]:
df = pd.read_json("hf://datasets/proj-persona/PersonaHub/persona.jsonl", lines=True)

In [5]:
print('N =', len(df))
df.head()

N = 200000


,persona
0,A Political Analyst specialized in El Salvador...
1,A legal advisor who understands the legal impl...
2,A maternal health advocate focused on raising ...
3,A school basketball team captain who believes ...
4,A determined basketball player who aspires to ...


In [6]:
# randomly sample 300 personas
df_sample = df.sample(n=300)
print('N =', len(df_sample))

N = 300


In [7]:
df_sample.to_json("../input_data/persona_hub.json")